<a href="https://colab.research.google.com/github/crystdang/DABC-Final-G5/blob/main/notebooks/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-01-26 02:17:17--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  6.18MB/s    in 0.2s    

2023-01-26 02:17:18 (6.18 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://uoft-dabc-final-g5.s3.us-east-2.amazonaws.com/Wellbeing_and_lifestyle_data_Kaggle.csv"
spark.sparkContext.addFile(url)
data_df = spark.read.csv(SparkFiles.get("Wellbeing_and_lifestyle_data_Kaggle.csv"), sep=",", header=True, inferSchema=True)

In [5]:
data_df.show()

+---------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+----------+------+-----------------------+
|Timestamp|FRUITS_VEGGIES|DAILY_STRESS|PLACES_VISITED|CORE_CIRCLE|SUPPORTING_OTHERS|SOCIAL_NETWORK|ACHIEVEMENT|DONATION|BMI_RANGE|TODO_COMPLETED|FLOW|DAILY_STEPS|LIVE_VISION|SLEEP_HOURS|LOST_VACATION|DAILY_SHOUTING|SUFFICIENT_INCOME|PERSONAL_AWARDS|TIME_FOR_PASSION|WEEKLY_MEDITATION|       AGE|GENDER|WORK_LIFE_BALANCE_SCORE|
+---------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+----------+------+-----------------------+
|   7/7/15|        

In [6]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dabc-final-g5.cnjpfxvbwyqz.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": "DataAnalytics", 
          "driver":"org.postgresql.Driver"}

In [7]:
# Write staging DataFrame to STG_WL table in RDS

data_df.write.jdbc(url=jdbc_url, table='STG_WL', mode=mode, properties=config)

In [8]:
data_df.columns

['Timestamp',
 'FRUITS_VEGGIES',
 'DAILY_STRESS',
 'PLACES_VISITED',
 'CORE_CIRCLE',
 'SUPPORTING_OTHERS',
 'SOCIAL_NETWORK',
 'ACHIEVEMENT',
 'DONATION',
 'BMI_RANGE',
 'TODO_COMPLETED',
 'FLOW',
 'DAILY_STEPS',
 'LIVE_VISION',
 'SLEEP_HOURS',
 'LOST_VACATION',
 'DAILY_SHOUTING',
 'SUFFICIENT_INCOME',
 'PERSONAL_AWARDS',
 'TIME_FOR_PASSION',
 'WEEKLY_MEDITATION',
 'AGE',
 'GENDER',
 'WORK_LIFE_BALANCE_SCORE']

In [9]:
data_df.printSchema()
# Timestamp and DAILY_STRESS to be converted

root
 |-- Timestamp: string (nullable = true)
 |-- FRUITS_VEGGIES: integer (nullable = true)
 |-- DAILY_STRESS: string (nullable = true)
 |-- PLACES_VISITED: integer (nullable = true)
 |-- CORE_CIRCLE: integer (nullable = true)
 |-- SUPPORTING_OTHERS: integer (nullable = true)
 |-- SOCIAL_NETWORK: integer (nullable = true)
 |-- ACHIEVEMENT: integer (nullable = true)
 |-- DONATION: integer (nullable = true)
 |-- BMI_RANGE: integer (nullable = true)
 |-- TODO_COMPLETED: integer (nullable = true)
 |-- FLOW: integer (nullable = true)
 |-- DAILY_STEPS: integer (nullable = true)
 |-- LIVE_VISION: integer (nullable = true)
 |-- SLEEP_HOURS: integer (nullable = true)
 |-- LOST_VACATION: integer (nullable = true)
 |-- DAILY_SHOUTING: integer (nullable = true)
 |-- SUFFICIENT_INCOME: integer (nullable = true)
 |-- PERSONAL_AWARDS: integer (nullable = true)
 |-- TIME_FOR_PASSION: integer (nullable = true)
 |-- WEEKLY_MEDITATION: integer (nullable = true)
 |-- AGE: string (nullable = true)
 |-- GE

In [10]:
data_df.describe().show()
# no null values

+-------+-----------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+-------------------+------------------+------------------+-----------------+------------+------+-----------------------+
|summary|  Timestamp|    FRUITS_VEGGIES|      DAILY_STRESS|    PLACES_VISITED|       CORE_CIRCLE|SUPPORTING_OTHERS|   SOCIAL_NETWORK|       ACHIEVEMENT|          DONATION|         BMI_RANGE|   TODO_COMPLETED|              FLOW|       DAILY_STEPS|      LIVE_VISION|       SLEEP_HOURS|     LOST_VACATION|    DAILY_SHOUTING|  SUFFICIENT_INCOME|   PERSONAL_AWARDS|  TIME_FOR_PASSION|WEEKLY_MEDITATION|         AGE|GENDER|WORK_LIFE_BALANCE_SCORE|
+-------+-----------+------------------+------------------+------------------+------------------+-----------------+-

In [11]:
# string type, confirm that it can be converted to date type
data_df.groupBy("Timestamp").count().show()

+-------------+-----+
|    Timestamp|count|
+-------------+-----+
|      9/29/15|   12|
|      1/22/16|    6|
|      10/4/17|   19|
|      7/15/18|    7|
|1/30/19 12:29|    1|
|2/26/19 15:43|    1|
| 3/14/19 6:15|    1|
| 4/20/19 3:51|    1|
| 4/25/19 8:04|    1|
|5/12/19 13:18|    1|
| 5/22/19 2:24|    1|
|5/26/19 13:40|    1|
|7/25/19 17:41|    1|
| 8/31/19 4:08|    1|
|9/20/19 17:42|    1|
| 11/3/19 7:45|    1|
|11/7/19 20:27|    1|
|11/9/19 17:44|    1|
| 1/18/20 7:59|    2|
|2/18/20 18:41|    1|
+-------------+-----+
only showing top 20 rows



In [12]:
from pyspark.sql.functions import regexp_replace
# remove timestamp
data_df = data_df.withColumn("DATE", regexp_replace("Timestamp", r'(\s\d{1,2}:\d{2})', ''))

In [13]:
# confirm timestamp is removed
data_df.groupBy("DATE").count().show()

+--------+-----+
|    DATE|count|
+--------+-----+
| 9/29/15|   12|
| 1/22/16|    6|
| 10/4/17|   19|
| 7/15/18|    7|
| 2/16/19|    8|
|  6/7/19|   12|
|  2/4/20|    7|
| 6/15/20|   10|
| 8/28/20|   10|
| 2/20/16|    8|
| 4/19/16|    8|
|  8/1/16|    4|
|11/10/16|    6|
|11/12/16|    3|
|  2/6/17|   13|
|11/22/17|    8|
| 7/30/18|    3|
| 5/19/19|   10|
|  7/5/19|    4|
| 7/26/19|    4|
+--------+-----+
only showing top 20 rows



In [14]:
# identify bad data
data_df.groupby("DAILY_STRESS").count().show()

+------------+-----+
|DAILY_STRESS|count|
+------------+-----+
|           3| 4398|
|           0|  676|
|           5| 2052|
|      1/1/00|    1|
|           1| 2478|
|           4| 2960|
|           2| 3407|
+------------+-----+



In [15]:
# remove bad data
clean_df = data_df.filter(data_df.DAILY_STRESS <= 5)
clean_df.groupBy("DAILY_STRESS").count().show()

+------------+-----+
|DAILY_STRESS|count|
+------------+-----+
|           3| 4398|
|           0|  676|
|           5| 2052|
|           1| 2478|
|           4| 2960|
|           2| 3407|
+------------+-----+



In [16]:
# adjust data type for DAILY_STRESS
clean_df = clean_df.withColumn("DAILY_STRESS_clean", clean_df["DAILY_STRESS"].cast('int')).drop("DAILY_STRESS").withColumnRenamed("DAILY_STRESS_clean", "DAILY_STRESS")
clean_df.printSchema()

root
 |-- Timestamp: string (nullable = true)
 |-- FRUITS_VEGGIES: integer (nullable = true)
 |-- PLACES_VISITED: integer (nullable = true)
 |-- CORE_CIRCLE: integer (nullable = true)
 |-- SUPPORTING_OTHERS: integer (nullable = true)
 |-- SOCIAL_NETWORK: integer (nullable = true)
 |-- ACHIEVEMENT: integer (nullable = true)
 |-- DONATION: integer (nullable = true)
 |-- BMI_RANGE: integer (nullable = true)
 |-- TODO_COMPLETED: integer (nullable = true)
 |-- FLOW: integer (nullable = true)
 |-- DAILY_STEPS: integer (nullable = true)
 |-- LIVE_VISION: integer (nullable = true)
 |-- SLEEP_HOURS: integer (nullable = true)
 |-- LOST_VACATION: integer (nullable = true)
 |-- DAILY_SHOUTING: integer (nullable = true)
 |-- SUFFICIENT_INCOME: integer (nullable = true)
 |-- PERSONAL_AWARDS: integer (nullable = true)
 |-- TIME_FOR_PASSION: integer (nullable = true)
 |-- WEEKLY_MEDITATION: integer (nullable = true)
 |-- AGE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- WORK_LIF

In [17]:
from pyspark.sql.functions import to_date
# adjust data type and reconfigure table
df = clean_df.select([to_date("DATE", "M/d/yy").alias("DATE"),
                          'FRUITS_VEGGIES',
                          'DAILY_STRESS',
                          'PLACES_VISITED',
                          'CORE_CIRCLE',
                          'SUPPORTING_OTHERS',
                          'SOCIAL_NETWORK',
                          'ACHIEVEMENT',
                          'DONATION',
                          'BMI_RANGE',
                          'TODO_COMPLETED',
                          'FLOW',
                          'DAILY_STEPS',
                          'LIVE_VISION',
                          'SLEEP_HOURS',
                          'LOST_VACATION',
                          'DAILY_SHOUTING',
                          'SUFFICIENT_INCOME',
                          'PERSONAL_AWARDS',
                          'TIME_FOR_PASSION',
                          'WEEKLY_MEDITATION',
                          'AGE',
                          'GENDER',
                          'WORK_LIFE_BALANCE_SCORE'])
df.show()

+----------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+----------+------+-----------------------+
|      DATE|FRUITS_VEGGIES|DAILY_STRESS|PLACES_VISITED|CORE_CIRCLE|SUPPORTING_OTHERS|SOCIAL_NETWORK|ACHIEVEMENT|DONATION|BMI_RANGE|TODO_COMPLETED|FLOW|DAILY_STEPS|LIVE_VISION|SLEEP_HOURS|LOST_VACATION|DAILY_SHOUTING|SUFFICIENT_INCOME|PERSONAL_AWARDS|TIME_FOR_PASSION|WEEKLY_MEDITATION|       AGE|GENDER|WORK_LIFE_BALANCE_SCORE|
+----------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+----------+------+-----------------------+
|2015-07-07|    

In [18]:
date_df = df.groupBy("DATE").count()
date_df.show()

+----------+-----+
|      DATE|count|
+----------+-----+
|2016-03-01|   12|
|2017-08-11|    3|
|2017-09-11|    7|
|2018-05-28|   10|
|2018-08-10|   10|
|2019-05-08|    7|
|2019-06-04|    7|
|2020-08-24|    8|
|2016-04-25|   14|
|2017-01-06|   12|
|2018-03-17|    3|
|2018-06-06|    1|
|2019-09-22|    5|
|2019-11-01|    8|
|2019-11-18|    7|
|2020-01-21|   11|
|2020-07-24|    9|
|2020-08-05|    6|
|2020-11-29|    5|
|2015-09-02|   26|
+----------+-----+
only showing top 20 rows



In [19]:
df.printSchema()

root
 |-- DATE: date (nullable = true)
 |-- FRUITS_VEGGIES: integer (nullable = true)
 |-- DAILY_STRESS: integer (nullable = true)
 |-- PLACES_VISITED: integer (nullable = true)
 |-- CORE_CIRCLE: integer (nullable = true)
 |-- SUPPORTING_OTHERS: integer (nullable = true)
 |-- SOCIAL_NETWORK: integer (nullable = true)
 |-- ACHIEVEMENT: integer (nullable = true)
 |-- DONATION: integer (nullable = true)
 |-- BMI_RANGE: integer (nullable = true)
 |-- TODO_COMPLETED: integer (nullable = true)
 |-- FLOW: integer (nullable = true)
 |-- DAILY_STEPS: integer (nullable = true)
 |-- LIVE_VISION: integer (nullable = true)
 |-- SLEEP_HOURS: integer (nullable = true)
 |-- LOST_VACATION: integer (nullable = true)
 |-- DAILY_SHOUTING: integer (nullable = true)
 |-- SUFFICIENT_INCOME: integer (nullable = true)
 |-- PERSONAL_AWARDS: integer (nullable = true)
 |-- TIME_FOR_PASSION: integer (nullable = true)
 |-- WEEKLY_MEDITATION: integer (nullable = true)
 |-- AGE: string (nullable = true)
 |-- GENDER: 

In [20]:
# define age groups
age_groups = df.groupBy("AGE").count()
age_groups.show()

+------------+-----+
|         AGE|count|
+------------+-----+
|  51 or more| 3390|
|    36 to 50| 4655|
|    21 to 35| 6107|
|Less than 20| 1819|
+------------+-----+



In [21]:
# arrange from high to low WLB score
high_low = df.orderBy(df["WORK_LIFE_BALANCE_SCORE"].desc())
high_low.show()

+----------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+------------+------+-----------------------+
|      DATE|FRUITS_VEGGIES|DAILY_STRESS|PLACES_VISITED|CORE_CIRCLE|SUPPORTING_OTHERS|SOCIAL_NETWORK|ACHIEVEMENT|DONATION|BMI_RANGE|TODO_COMPLETED|FLOW|DAILY_STEPS|LIVE_VISION|SLEEP_HOURS|LOST_VACATION|DAILY_SHOUTING|SUFFICIENT_INCOME|PERSONAL_AWARDS|TIME_FOR_PASSION|WEEKLY_MEDITATION|         AGE|GENDER|WORK_LIFE_BALANCE_SCORE|
+----------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+------------+------+-----------------------+
|2015-10-1

In [22]:
# arrange from low to high WLB score
low_high = df.orderBy(df["WORK_LIFE_BALANCE_SCORE"])
low_high.show()

+----------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+------------+------+-----------------------+
|      DATE|FRUITS_VEGGIES|DAILY_STRESS|PLACES_VISITED|CORE_CIRCLE|SUPPORTING_OTHERS|SOCIAL_NETWORK|ACHIEVEMENT|DONATION|BMI_RANGE|TODO_COMPLETED|FLOW|DAILY_STEPS|LIVE_VISION|SLEEP_HOURS|LOST_VACATION|DAILY_SHOUTING|SUFFICIENT_INCOME|PERSONAL_AWARDS|TIME_FOR_PASSION|WEEKLY_MEDITATION|         AGE|GENDER|WORK_LIFE_BALANCE_SCORE|
+----------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+------------+------+-----------------------+
|2016-10-2

In [23]:
# WLB score average, min, max
from pyspark.sql.functions import avg, min, max
df.select(avg("WORK_LIFE_BALANCE_SCORE"), min("WORK_LIFE_BALANCE_SCORE"), max("WORK_LIFE_BALANCE_SCORE")).show()

+----------------------------+----------------------------+----------------------------+
|avg(WORK_LIFE_BALANCE_SCORE)|min(WORK_LIFE_BALANCE_SCORE)|max(WORK_LIFE_BALANCE_SCORE)|
+----------------------------+----------------------------+----------------------------+
|             666.75051029992|                       480.0|                       820.2|
+----------------------------+----------------------------+----------------------------+



In [24]:
df.groupBy("GENDER").count().show()

+------+-----+
|GENDER|count|
+------+-----+
|Female| 9858|
|  Male| 6113|
+------+-----+



In [25]:
# female participants
female_df = df.filter(df["GENDER"] == "Female")
female_df.show()

+----------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+------------+------+-----------------------+
|      DATE|FRUITS_VEGGIES|DAILY_STRESS|PLACES_VISITED|CORE_CIRCLE|SUPPORTING_OTHERS|SOCIAL_NETWORK|ACHIEVEMENT|DONATION|BMI_RANGE|TODO_COMPLETED|FLOW|DAILY_STEPS|LIVE_VISION|SLEEP_HOURS|LOST_VACATION|DAILY_SHOUTING|SUFFICIENT_INCOME|PERSONAL_AWARDS|TIME_FOR_PASSION|WEEKLY_MEDITATION|         AGE|GENDER|WORK_LIFE_BALANCE_SCORE|
+----------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+------------+------+-----------------------+
|2015-07-0

In [26]:
# male participants
male_df = df.filter(df["GENDER"] == "Male")
male_df.show()

+----------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+----------+------+-----------------------+
|      DATE|FRUITS_VEGGIES|DAILY_STRESS|PLACES_VISITED|CORE_CIRCLE|SUPPORTING_OTHERS|SOCIAL_NETWORK|ACHIEVEMENT|DONATION|BMI_RANGE|TODO_COMPLETED|FLOW|DAILY_STEPS|LIVE_VISION|SLEEP_HOURS|LOST_VACATION|DAILY_SHOUTING|SUFFICIENT_INCOME|PERSONAL_AWARDS|TIME_FOR_PASSION|WEEKLY_MEDITATION|       AGE|GENDER|WORK_LIFE_BALANCE_SCORE|
+----------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+----------+------+-----------------------+
|2015-07-08|    

In [27]:
from pyspark.sql.functions import col
final_df = df.drop(col("GENDER")).drop(col("DATE"))
final_df.show()

+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+----------+-----------------------+
|FRUITS_VEGGIES|DAILY_STRESS|PLACES_VISITED|CORE_CIRCLE|SUPPORTING_OTHERS|SOCIAL_NETWORK|ACHIEVEMENT|DONATION|BMI_RANGE|TODO_COMPLETED|FLOW|DAILY_STEPS|LIVE_VISION|SLEEP_HOURS|LOST_VACATION|DAILY_SHOUTING|SUFFICIENT_INCOME|PERSONAL_AWARDS|TIME_FOR_PASSION|WEEKLY_MEDITATION|       AGE|WORK_LIFE_BALANCE_SCORE|
+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+----------+-----------------------+
|             3|           2|             2|          5|              

In [28]:
import pandas as pd
pandas_df = df.toPandas()

In [29]:
pandas_df.head(10)

,DATE,FRUITS_VEGGIES,DAILY_STRESS,PLACES_VISITED,CORE_CIRCLE,SUPPORTING_OTHERS,SOCIAL_NETWORK,ACHIEVEMENT,DONATION,BMI_RANGE,...,SLEEP_HOURS,LOST_VACATION,DAILY_SHOUTING,SUFFICIENT_INCOME,PERSONAL_AWARDS,TIME_FOR_PASSION,WEEKLY_MEDITATION,AGE,GENDER,WORK_LIFE_BALANCE_SCORE
0,2015-07-07,3,2,2,5,0,5,2,0,1,...,7,5,5,1,4,0,5,36 to 50,Female,609.5
1,2015-07-07,2,3,4,3,8,10,5,2,2,...,8,2,2,2,3,2,6,36 to 50,Female,655.6
2,2015-07-07,2,3,3,4,4,10,3,2,2,...,8,10,2,2,4,8,3,36 to 50,Female,631.6
3,2015-07-07,3,3,10,3,10,7,2,5,2,...,5,7,5,1,5,2,0,51 or more,Female,622.7
4,2015-07-07,5,1,3,3,10,4,2,4,2,...,7,0,0,2,8,1,5,51 or more,Female,663.9
5,2015-07-08,3,2,3,9,10,10,2,3,1,...,8,0,2,2,10,8,3,51 or more,Female,722.3
6,2015-07-08,4,2,10,6,10,10,3,5,2,...,7,10,0,2,10,8,10,51 or more,Male,727.2
7,2015-07-09,3,4,5,3,5,7,4,0,1,...,6,0,2,2,8,2,2,21 to 35,Female,676.2
8,2015-07-09,5,3,6,4,3,3,5,4,1,...,10,0,2,2,10,3,10,21 to 35,Female,702.8
9,2015-07-10,4,4,2,6,10,10,0,4,2,...,6,0,0,1,3,8,1,51 or more,Female,634.4


In [30]:
# Write transformed DataFrame to TRF_WL table in RDS

df.write.jdbc(url=jdbc_url, table='TRF_WL', mode=mode, properties=config)

In [31]:
# disect totals in each variable
final_df.groupBy("FRUITS_VEGGIES").count().show()

+--------------+-----+
|FRUITS_VEGGIES|count|
+--------------+-----+
|             1| 2421|
|             3| 3736|
|             5| 3141|
|             4| 2551|
|             2| 3570|
|             0|  552|
+--------------+-----+



In [32]:
final_df.groupBy("DAILY_STRESS").count().show()

+------------+-----+
|DAILY_STRESS|count|
+------------+-----+
|           1| 2478|
|           3| 4398|
|           5| 2052|
|           4| 2960|
|           2| 3407|
|           0|  676|
+------------+-----+



In [33]:
final_df.groupBy("PLACES_VISITED").count().show()

+--------------+-----+
|PLACES_VISITED|count|
+--------------+-----+
|             1| 1252|
|             6| 1136|
|             3| 1840|
|             5| 1862|
|             9|  258|
|             4| 1503|
|             8|  881|
|             7|  878|
|            10| 3558|
|             2| 1787|
|             0| 1016|
+--------------+-----+



In [34]:
final_df.groupBy("CORE_CIRCLE").count().show()

+-----------+-----+
|CORE_CIRCLE|count|
+-----------+-----+
|          1|  719|
|          6| 1702|
|          3| 1941|
|          5| 2410|
|          9|  368|
|          4| 2151|
|          8| 1090|
|          7| 1141|
|         10| 2784|
|          2| 1353|
|          0|  312|
+-----------+-----+



In [35]:
final_df.groupBy("SUPPORTING_OTHERS").count().show()

+-----------------+-----+
|SUPPORTING_OTHERS|count|
+-----------------+-----+
|                1|  883|
|                6| 1119|
|                3| 1835|
|                5| 1915|
|                9|  294|
|                4| 1646|
|                8|  917|
|                7| 1053|
|               10| 3993|
|                2| 1576|
|                0|  740|
+-----------------+-----+



In [36]:
final_df.groupBy("SOCIAL_NETWORK").count().show()

+--------------+-----+
|SOCIAL_NETWORK|count|
+--------------+-----+
|             1|  556|
|             6| 1219|
|             3| 1593|
|             5| 1912|
|             9|  336|
|             4| 1560|
|             8| 1011|
|             7|  972|
|            10| 5455|
|             2| 1241|
|             0|  116|
+--------------+-----+



In [37]:
final_df.groupBy("ACHIEVEMENT").count().show()

+-----------+-----+
|ACHIEVEMENT|count|
+-----------+-----+
|          1| 1595|
|          6| 1145|
|          3| 2538|
|          5| 2030|
|          9|  240|
|          4| 1838|
|          8|  693|
|          7|  814|
|         10| 1170|
|          2| 2606|
|          0| 1302|
+-----------+-----+



In [38]:
final_df.groupBy("DONATION").count().show()

+--------+-----+
|DONATION|count|
+--------+-----+
|       1| 2668|
|       3| 2210|
|       5| 4760|
|       4| 1300|
|       2| 2533|
|       0| 2500|
+--------+-----+



In [39]:
final_df.groupBy("BMI_RANGE").count().show()

+---------+-----+
|BMI_RANGE|count|
+---------+-----+
|        1| 9413|
|        2| 6558|
+---------+-----+



In [40]:
final_df.groupBy("TODO_COMPLETED").count().show()

+--------------+-----+
|TODO_COMPLETED|count|
+--------------+-----+
|             1|  599|
|             6| 1665|
|             3| 1414|
|             5| 2092|
|             9| 1079|
|             4| 1326|
|             8| 2587|
|             7| 2553|
|            10| 1083|
|             2| 1033|
|             0|  540|
+--------------+-----+



In [41]:
final_df.groupBy("FLOW").count().show()

+----+-----+
|FLOW|count|
+----+-----+
|   1| 2983|
|   6| 1028|
|   3| 2485|
|   5| 1483|
|   9|  154|
|   4| 1833|
|   8|  551|
|   7|  584|
|  10|  338|
|   2| 3202|
|   0| 1330|
+----+-----+



In [42]:
final_df.groupBy("DAILY_STEPS").count().show()

+-----------+-----+
|DAILY_STEPS|count|
+-----------+-----+
|          1| 1251|
|          6| 1597|
|          3| 1567|
|          5| 2161|
|          9|  706|
|          4| 1504|
|          8| 1543|
|          7| 1431|
|         10| 2700|
|          2| 1511|
+-----------+-----+



In [43]:
final_df.groupBy("LIVE_VISION").count().show()

+-----------+-----+
|LIVE_VISION|count|
+-----------+-----+
|          1| 2485|
|          6|  461|
|          3| 1767|
|          5| 2544|
|          9|  113|
|          4|  978|
|          8|  351|
|          7|  413|
|         10| 2168|
|          2| 2173|
|          0| 2518|
+-----------+-----+



In [44]:
final_df.groupBy("SLEEP_HOURS").count().show()

+-----------+-----+
|SLEEP_HOURS|count|
+-----------+-----+
|          1|   18|
|          6| 3396|
|          3|   49|
|          5| 1025|
|          9|  987|
|          4|  252|
|          8| 4324|
|          7| 5566|
|         10|  333|
|          2|   21|
+-----------+-----+



In [45]:
final_df.groupBy("LOST_VACATION").count().show()

+-------------+-----+
|LOST_VACATION|count|
+-------------+-----+
|            1|  600|
|            6|  343|
|            3|  807|
|            5| 1240|
|            9|  123|
|            4|  671|
|            8|  373|
|            7|  587|
|           10| 2238|
|            2|  874|
|            0| 8115|
+-------------+-----+



In [46]:
final_df.groupBy("DAILY_SHOUTING").count().show()

+--------------+-----+
|DAILY_SHOUTING|count|
+--------------+-----+
|             1| 3726|
|             6|  545|
|             3| 2101|
|             5| 1252|
|             9|  145|
|             4| 1255|
|             8|  378|
|             7|  669|
|            10|  785|
|             2| 2685|
|             0| 2430|
+--------------+-----+



In [47]:
final_df.groupBy("SUFFICIENT_INCOME").count().show()

+-----------------+-----+
|SUFFICIENT_INCOME|count|
+-----------------+-----+
|                1| 4329|
|                2|11642|
+-----------------+-----+



In [48]:
final_df.groupBy("PERSONAL_AWARDS").count().show()

+---------------+-----+
|PERSONAL_AWARDS|count|
+---------------+-----+
|              1|  713|
|              6| 1344|
|              3| 1881|
|              5| 2210|
|              9|  335|
|              4| 1733|
|              8|  946|
|              7| 1118|
|             10| 3764|
|              2| 1382|
|              0|  545|
+---------------+-----+



In [49]:
final_df.groupBy("TIME_FOR_PASSION").count().show()

+----------------+-----+
|TIME_FOR_PASSION|count|
+----------------+-----+
|               1| 3285|
|               6|  998|
|               3| 1962|
|               5| 1229|
|               9|  193|
|               4| 1504|
|               8|  905|
|               7|  635|
|              10|  682|
|               2| 2781|
|               0| 1797|
+----------------+-----+



In [50]:
final_df.groupBy("WEEKLY_MEDITATION").count().show()

+-----------------+-----+
|WEEKLY_MEDITATION|count|
+-----------------+-----+
|                1|  701|
|                6| 1043|
|                3| 1487|
|                5| 1977|
|                9|  441|
|                4| 1310|
|                8|  993|
|                7| 2275|
|               10| 4285|
|                2| 1162|
|                0|  297|
+-----------------+-----+



In [51]:
final_df.groupBy("WORK_LIFE_BALANCE_SCORE").count().sort(col("WORK_LIFE_BALANCE_SCORE").desc()).show()

+-----------------------+-----+
|WORK_LIFE_BALANCE_SCORE|count|
+-----------------------+-----+
|                  820.2|    1|
|                  818.3|    1|
|                  816.4|    1|
|                  814.5|    2|
|                  804.3|    2|
|                  802.6|    1|
|                  801.1|    1|
|                  799.4|    1|
|                  799.2|    1|
|                  799.0|    1|
|                  797.7|    2|
|                  797.5|    1|
|                  795.8|    1|
|                  795.4|    1|
|                  794.8|    1|
|                  794.6|    1|
|                  794.3|    1|
|                  792.6|    1|
|                  792.4|    1|
|                  790.9|    2|
+-----------------------+-----+
only showing top 20 rows



In [52]:
# WLB score scale, excellent, good and bad
excellent = final_df.filter(col("WORK_LIFE_BALANCE_SCORE") >= 700)
excellent.count()

3816

In [53]:
good = final_df.filter((col("WORK_LIFE_BALANCE_SCORE") > 550) & (col("WORK_LIFE_BALANCE_SCORE") < 700))
good.count()

12081

In [54]:
bad = final_df.filter(col("WORK_LIFE_BALANCE_SCORE") <= 550)
bad.count()

74

In [55]:
# total for "bad" too low, merge with good and renamed as "needs improvement"
needs_improvement = final_df.filter(col("WORK_LIFE_BALANCE_SCORE") < 700)
needs_improvement.count()

12155